# MNIST ANN
Based on [dm-haiku/mnist.py](https://github.com/deepmind/dm-haiku/blob/main/examples/mnist.py)

In [ ]:
%pip install \
    git+https://github.com/deepmind/dm-haiku@v0.0.4 \
    git+https://github.com/deepmind/optax@v0.0.9

In [3]:
from functools import partial
import haiku as hk

import jax
from jax import jit, vmap, grad
from jax import random
import jax.lax as lax
import jax.nn as nn
import jax.numpy as np

import optax

import tensorflow_datasets as tfds

In [4]:
rng = random.PRNGKey(42)

In [5]:
def ravel_tree(tree):
    return np.concatenate(list(map(np.ravel, jax.tree_leaves(tree))))
ravel_tree((np.array([1, 2, 3]), np.array([[4, 5], [6, 7]])))

DeviceArray([1, 2, 3, 4, 5, 6, 7], dtype=int32)

In [ ]:
def load_dataset(split, *, is_training, batch_size):
    ds = tfds.load("mnist:3.*.*", split=split).cache().repeat()
    if is_training:
        ds = ds.shuffle(10 * batch_size, seed=0)
    ds = ds.batch(batch_size)
    return iter(tfds.as_numpy(ds))
train = load_dataset("train", is_training=True, batch_size=128)
train_eval = load_dataset("train", is_training=False, batch_size=1024)
test_eval = load_dataset("test", is_training=False, batch_size=1024)
batch = next(train)
batch['image'].shape, batch['label'].shape

In [7]:
label_count = 10

@hk.without_apply_rng
@hk.transform
def model(batch):
    x = batch["image"].astype(np.float32) / 255.
    mlp = hk.Sequential([
          hk.Flatten(),
          hk.Linear(300), nn.relu,
          hk.Linear(100), nn.relu,
          hk.Linear(label_count),
    ])
    return mlp(x)
print(hk.experimental.tabulate(model, columns=['module', 'input', 'output', 'params_size'])(batch))

+----------------------------+------------------+--------------+---------------+
| Module                     | Input            | Output       |   Param count |
+============================+==================+==============+===============+
| sequential (Sequential)    | f32[128,28,28,1] | f32[128,10]  |       266,610 |
+----------------------------+------------------+--------------+---------------+
| flatten (Flatten)          | f32[128,28,28,1] | f32[128,784] |             0 |
|  └ sequential (Sequential) |                  |              |               |
+----------------------------+------------------+--------------+---------------+
| linear (Linear)            | f32[128,784]     | f32[128,300] |       235,500 |
|  └ sequential (Sequential) |                  |              |               |
+----------------------------+------------------+--------------+---------------+
| linear_1 (Linear)          | f32[128,300]     | f32[128,100] |        30,100 |
|  └ sequential (Sequential)

In [8]:
def loss(params, batch):
    logits = model.apply(params, batch)
    labels = nn.one_hot(batch['label'], label_count)
    l2_loss = np.sum(optax.l2_loss(ravel_tree(params)))
    softmax_xent = optax.softmax_cross_entropy(logits, labels)
    softmax_xent = np.mean(softmax_xent)
    softmax_xent = softmax_xent + 1e-4 * l2_loss
    return softmax_xent
rng, r = random.split(rng)
weights = average_weights= model.init(r, next(train))
loss(weights, batch)

DeviceArray(2.3166852, dtype=float32)

In [9]:
@jax.jit
def accuracy(weights, batch):
    predictions = model.apply(weights, batch)
    return np.mean(np.argmax(predictions, axis=-1) == batch["label"])
accuracy(weights, batch)

DeviceArray(0.125, dtype=float32)

In [10]:
optimizer = optax.adam(1e-3)
optimizer_state = optimizer.init(weights)

In [11]:
@jax.jit
def update(weights, optimizer_state, batch):
    loss_grads = grad(loss)(weights, batch)
    optimizer_updates, optimizer_state = optimizer.update(loss_grads, optimizer_state)
    weights = optax.apply_updates(weights, optimizer_updates)
    return weights, optimizer_state
np.mean(np.abs(ravel_tree(update(weights, optimizer_state, batch)[0])))

DeviceArray(0.02740272, dtype=float32)

In [12]:
@jit
def ema_update(weights, average_weights):
    return optax.incremental_update(weights, average_weights, step_size=0.001)
np.mean(np.abs(ravel_tree(ema_update(weights, average_weights))))

DeviceArray(0.0277532, dtype=float32)

In [13]:
for step in range(2000):
    if step % 100 == 0:
        train_accuracy = accuracy(average_weights, next(train_eval))
        test_accuracy = accuracy(average_weights, next(test_eval))
        print(f"[Step {step}] Train / Test accuracy: {train_accuracy:.3f} / {test_accuracy:.3f}.")

    weights, optimizer_state = update(weights, optimizer_state, next(train))
    average_weights = ema_update(weights, average_weights)

[Step 0] Train / Test accuracy: 0.101 / 0.105.
[Step 100] Train / Test accuracy: 0.463 / 0.463.
[Step 200] Train / Test accuracy: 0.721 / 0.701.
[Step 300] Train / Test accuracy: 0.823 / 0.826.
[Step 400] Train / Test accuracy: 0.885 / 0.898.
[Step 500] Train / Test accuracy: 0.912 / 0.918.
[Step 600] Train / Test accuracy: 0.934 / 0.926.
[Step 700] Train / Test accuracy: 0.939 / 0.946.
[Step 800] Train / Test accuracy: 0.963 / 0.954.
[Step 900] Train / Test accuracy: 0.961 / 0.956.
[Step 1000] Train / Test accuracy: 0.964 / 0.951.
[Step 1100] Train / Test accuracy: 0.961 / 0.957.
[Step 1200] Train / Test accuracy: 0.978 / 0.959.
[Step 1300] Train / Test accuracy: 0.969 / 0.961.
[Step 1400] Train / Test accuracy: 0.977 / 0.971.
[Step 1500] Train / Test accuracy: 0.982 / 0.969.
[Step 1600] Train / Test accuracy: 0.974 / 0.971.
[Step 1700] Train / Test accuracy: 0.982 / 0.983.
[Step 1800] Train / Test accuracy: 0.988 / 0.977.
[Step 1900] Train / Test accuracy: 0.989 / 0.979.
